In [1]:
import pandas as pd
import numpy as np
import re


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
np.random.seed(500)

In [2]:
#uploading Mashins cleaned File in order to create the support vector machine
filename='CleanedDatasetForML.csv'
data=pd.read_csv(filename)
print(data.shape[0])

25108


In [3]:
#assigning the cleaned plots dataframe into a python variable and splitting the genere using split str
clean_plot=data["Clean_Plot"]
data['GenreSplit']=data['Selected_Genre'].str.split('|')
genres=data['GenreSplit']


In [4]:

###Removing any blank rows
data["Clean_Plot"].dropna(inplace=True)
##changes all the text to lower case from higher case for tokenization
data['Clean_Plot'] = [entry.lower() for entry in data['Clean_Plot']]
#cleaning n's Usernames, IP and http links
data['Clean_Plot'] = data['Clean_Plot'].map(lambda x: re.sub('\\n',' ',str(x)))
data['Clean_Plot'] = data['Clean_Plot'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
data['Clean_Plot'] = data['Clean_Plot'].map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", '', str(x)))
data['Clean_Plot'] = data['Clean_Plot'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))

In [5]:
#creates a new column called count which has 1 as a default value. This will be used later on in the code
data['Count']=1
#assigns x to the cleaned plot column 
x=data['Clean_Plot']
#counts the number of times a certain genre appears within a row. For example row one has only action genre so it will 
#label action as 1 and every other genre as 0 in order to classify. If a row has mutiply genres it will assign 1 to 
#those specific genres and the rest will be classified as 0
movies_count=data[['Selected_Genre','Count']].groupby(['Selected_Genre'],as_index=False).count()
# changes the counts from horizantal to vertical
movies_count=data['Selected_Genre'].str.get_dummies()
y=movies_count
y.head(15)

,action,animation,biography,black,children,comedy,crime,documentary,drama,fantasy,fiction,history,musical,mystery,romance,science,series,thriller
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [6]:
#Splits the data set into train and test sets by doing a 80 :20 split as the test size is 0.2
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=0)

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(len(y.columns))
type(X_train)

(20086,)
(20086, 18)
(5022,)
(5022, 18)
18


pandas.core.series.Series

In [8]:
#implementing the word_coversion algorithm
word_conversion = TfidfVectorizer(stop_words='english', sublinear_tf=True,strip_accents='unicode',analyzer='word',
    token_pattern=r'\w{2,}',  #vectorize 2-character words or more
    ngram_range=(1, 1))
##converts the words Term Frequency-Inverse Document Frequency which is a way of converting strings to unique numerals 
#in order to predict the strings
word_conversion.fit(X_train)
X_train_word_features = word_conversion.transform(X_train)
X_test_word_features = word_conversion.transform(X_test)
type(X_train_word_features)

scipy.sparse.csr.csr_matrix

In [9]:
#X_train_word_features.toarray()

In [10]:
#displayes the words that have converted
word_conversion.inverse_transform(X_train_word_features[0].toarray())

[array(['aboard', 'abu', 'alerted', 'allowed', 'asks', 'assailant',
        'attempted', 'away', 'bay', 'beautiful', 'better', 'body',
        'bucket', 'buys', 'cairo', 'catch', 'catches', 'chases',
        'children', 'coaxes', 'cold', 'complies', 'crate', 'crowd',
        'darkness', 'dead', 'death', 'does', 'donned', 'drink', 'drinks',
        'engaged', 'escape', 'fantasy', 'friend', 'garment', 'gets',
        'gives', 'goes', 'handicap', 'hannuma', 'head', 'hide', 'hides',
        'holding', 'home', 'husky', 'identifies', 'illegally', 'impending',
        'incriminating', 'inspired', 'instead', 'job', 'kill', 'knife',
        'lame', 'little', 'local', 'lose', 'lot', 'luggage', 'lure',
        'madness', 'man', 'marry', 'men', 'mild', 'murder', 'near', 'news',
        'newspapers', 'newsstand', 'nonetheless', 'notice', 'obsessed',
        'obsession', 'organize', 'owner', 'pick', 'pin', 'pity', 'plots',
        'policemen', 'porter', 'proposes', 'putting', 'qinawi', 'rail',
     

In [11]:
print(X_train_word_features)

  (0, 99049)	0.03058351866402017
  (0, 98653)	0.06452137725723152
  (0, 97996)	0.05985081806827504
  (0, 97877)	0.07456686560164993
  (0, 97832)	0.07453827885855722
  (0, 97815)	0.05699511230139953
  (0, 96600)	0.07718206966516537
  (0, 96197)	0.06824994585694766
  (0, 96031)	0.05514850080380424
  (0, 95133)	0.046355704431658
  (0, 94922)	0.05727874706621601
  (0, 94542)	0.08752027797153462
  (0, 93226)	0.09218064640475974
  (0, 92880)	0.12704832889455892
  (0, 92877)	0.06228772477346144
  (0, 92312)	0.05168606538072378
  (0, 91775)	0.03634334326042135
  (0, 91463)	0.03912511144822489
  (0, 91459)	0.06893662204125688
  (0, 91348)	0.11565167275111929
  (0, 91326)	0.05035093382653109
  (0, 90623)	0.07747024271097679
  (0, 88379)	0.02751852936496373
  (0, 87467)	0.02790972116496228
  (0, 87454)	0.038566246974537095
  :	:
  (20085, 19101)	0.07441735732780333
  (20085, 18681)	0.07075810719874728
  (20085, 18552)	0.12406227407574966
  (20085, 18549)	0.07921696070823149
  (20085, 18499)	0.105

In [12]:
word_conversion.inverse_transform(X_train_word_features[0].toarray())
print('The amount of words contained is .'.format(len(X_train_word_features[0].toarray()[0])))

The amount of words contained is .


In [13]:
#assiging the support vector machine classifier as a one vs rest classifier
linearSVC=OneVsRestClassifier(LinearSVC(), n_jobs=1)
#creating two columns to show the genres and the accuracy of prediction retlated to these genres
svc=pd.DataFrame(columns=['Genre', 'accuracy'])

In [14]:
#creating the different genre names in-order for the for loop to iterate over
class_names = ['action','animation', 'biography','black','children','comedy','crime','documentary','drama','fantasy','fiction','history','musical','mystery','romance','science','series','thriller']

In [15]:
#Fit the training data to the support vector machine in order to test the data using the test data set
#and creating a classification report and a confusion matrix
i=0
for genre in class_names:
    linearSVC.fit(X_train_word_features,y_train[genre])
    print("genre: "+ genre)
    scores = cross_val_score(linearSVC, X_train_word_features, y_train[genre], cv=5)
    print(scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    prediction = linearSVC.predict(X_test_word_features)
    svc.loc[i,'Genre'] = genre
    svc.loc[i,'accuracy'] = accuracy_score(y_test[genre], prediction)
    print(confusion_matrix(y_test[genre], prediction))
    print(classification_report(y_test[genre], prediction))
    i=i+1

genre: action
[0.88850174 0.8802589  0.88747822 0.88000996 0.88996764]
Accuracy: 0.89 (+/- 0.01)
[[4075  154]
 [ 391  402]]
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      4229
           1       0.72      0.51      0.60       793

    accuracy                           0.89      5022
   macro avg       0.82      0.74      0.77      5022
weighted avg       0.88      0.89      0.88      5022

genre: animation
[0.981334   0.98083147 0.98132935 0.98307194 0.98033358]
Accuracy: 0.98 (+/- 0.00)
[[4863    6]
 [  85   68]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      4869
           1       0.92      0.44      0.60       153

    accuracy                           0.98      5022
   macro avg       0.95      0.72      0.79      5022
weighted avg       0.98      0.98      0.98      5022

genre: biography
[0.98830264 0.98854867 0.98829973 0.98829973 0.98829973]
Accuracy: 0.99 (+/- 0

/Users/macbookpro/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


genre: drama
[0.75261324 0.75777944 0.74607916 0.74831964 0.7403535 ]
Accuracy: 0.75 (+/- 0.01)
[[3192  460]
 [ 772  598]]
              precision    recall  f1-score   support

           0       0.81      0.87      0.84      3652
           1       0.57      0.44      0.49      1370

    accuracy                           0.75      5022
   macro avg       0.69      0.66      0.67      5022
weighted avg       0.74      0.75      0.74      5022

genre: fantasy
[0.99104032 0.99153597 0.99153597 0.99128703 0.99128703]
Accuracy: 0.99 (+/- 0.00)
[[4985    2]
 [  35    0]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      4987
           1       0.00      0.00      0.00        35

    accuracy                           0.99      5022
   macro avg       0.50      0.50      0.50      5022
weighted avg       0.99      0.99      0.99      5022

genre: fiction
[0.98058736 0.98033358 0.98257406 0.98083147 0.98182723]
Accuracy: 0.98 (+/- 0.00)


/Users/macbookpro/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


genre: musical
[0.97436536 0.97311426 0.97411003 0.97087379 0.97411003]
Accuracy: 0.97 (+/- 0.00)
[[4858   14]
 [ 120   30]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      4872
           1       0.68      0.20      0.31       150

    accuracy                           0.97      5022
   macro avg       0.83      0.60      0.65      5022
weighted avg       0.97      0.97      0.97      5022

genre: mystery
[0.98830264 0.98805078 0.98780184 0.98780184 0.98805078]
Accuracy: 0.99 (+/- 0.00)
[[4957    1]
 [  57    7]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4958
           1       0.88      0.11      0.19        64

    accuracy                           0.99      5022
   macro avg       0.93      0.55      0.59      5022
weighted avg       0.99      0.99      0.98      5022

genre: romance
[0.92259831 0.92606423 0.92332587 0.92432163 0.92730894]
Accuracy: 0.92 (+/- 0.00

In [16]:
#the print of our the accuracies
svc

,Genre,accuracy
0,action,0.891477
1,animation,0.98188
2,biography,0.987853
3,black,0.986858
4,children,0.985066
5,comedy,0.820191
6,crime,0.94763
7,documentary,0.995619
8,drama,0.754679
9,fantasy,0.992632


In [17]:
svc.to_csv('SVMModelAccuracy.csv',sep=',')